In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-10 18:36:55,795 INFO: Initializing external client
2025-05-10 18:36:55,795 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 18:36:56,636 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Fetching data from 2025-04-11 22:36:55.794434+00:00 to 2025-05-10 21:36:55.794434+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.40s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-04-12 18:00:00+00:00,HB101,15
1,2025-04-12 19:00:00+00:00,HB101,0
2,2025-04-12 20:00:00+00:00,HB101,0
3,2025-04-12 21:00:00+00:00,HB101,0
4,2025-04-12 22:00:00+00:00,HB101,0
...,...,...,...
55854,2025-05-10 14:00:00+00:00,JC116,0
55855,2025-05-10 15:00:00+00:00,JC116,0
55856,2025-05-10 16:00:00+00:00,JC116,0
55857,2025-05-10 17:00:00+00:00,JC116,0


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55859 entries, 0 to 55858
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_hour         55859 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  55859 non-null  object                 
 2   rides               55859 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(1), object(1)
memory usage: 1.1+ MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55859 entries, 0 to 55858
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         55859 non-null  datetime64[us]
 1   pickup_location_id  55859 non-null  object        
 2   rides               55859 non-null  int32         
dtypes: datetime64[us](1), int32(1), object(1)
memory usage: 1.1+ MB


In [11]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=2)

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,JC108,2025-04-22 08:00:00
1,0,0,0,18,0,0,0,0,0,22,...,0,0,0,0,0,0,0,27,JC009,2025-05-10 11:00:00
2,0,0,0,0,0,0,0,0,0,6,...,0,0,0,0,0,1,0,0,HB301,2025-05-07 11:00:00
3,0,0,10,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,JC075,2025-05-02 14:00:00
4,2,0,0,1,0,0,0,0,5,0,...,0,0,0,0,0,6,7,0,JC053,2025-04-27 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,JC020,2025-05-01 12:00:00
79,0,0,0,0,0,23,0,0,0,0,...,0,0,0,0,18,10,0,12,JC098,2025-04-20 02:00:00
80,0,0,0,0,0,0,0,0,9,0,...,0,0,0,0,0,0,0,0,HB305,2025-04-16 06:00:00
81,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,JC105,2025-05-06 20:00:00


In [13]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-05-10 18:39:32,652 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 18:39:32,655 INFO: Initializing external client
2025-05-10 18:39:32,655 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 18:39:33,383 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000
Fetching data from 2025-04-11 22:39:32.652745+00:00 to 2025-05-10 21:39:32.652745+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.89s) 


In [14]:
current_date

Timestamp('2025-05-10 22:39:32.652745+0000', tz='Etc/UTC')

In [15]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,15,0,0,0,0,0,10,0,0,0,...,0,0,26,0,0,0,0,0,HB101,2025-05-10 18:00:00
1,27,0,0,0,0,0,11,0,0,0,...,0,0,69,0,0,0,0,0,HB102,2025-05-10 18:00:00
2,6,0,0,0,0,0,11,0,0,0,...,0,0,30,0,0,0,0,0,HB103,2025-05-10 18:00:00
3,17,0,0,0,0,0,8,0,0,0,...,0,0,40,0,0,0,0,0,HB105,2025-05-10 18:00:00
4,0,0,0,0,0,0,1,0,0,0,...,0,0,42,0,0,0,0,0,HB201,2025-05-10 18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,JC108,2025-05-10 18:00:00
79,9,0,0,0,0,0,9,0,0,0,...,0,0,23,0,0,0,0,0,JC109,2025-05-10 18:00:00
80,2,0,0,0,0,0,1,0,0,0,...,0,0,21,0,0,0,0,0,JC110,2025-05-10 18:00:00
81,15,0,0,0,0,0,11,0,0,0,...,0,0,47,0,0,0,0,0,JC115,2025-05-10 18:00:00


In [16]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-10 18:40:14,086 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 18:40:14,088 INFO: Initializing external client
2025-05-10 18:40:14,088 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 18:40:14,744 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000


Downloading: 0.000%|          | 0/309419 elapsed<00:00 remaining<?

In [17]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [18]:
predictions

,pickup_location_id,predicted_demand
0,HB101,29.0
1,HB102,48.0
2,HB103,23.0
3,HB105,20.0
4,HB201,18.0
...,...,...
78,JC108,3.0
79,JC109,22.0
80,JC110,10.0
81,JC115,36.0


In [19]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array(['HB102', 'JC115', 'HB101', 'JC116', 'HB103', 'JC109', 'JC066',
       'JC008', 'HB105', 'HB603'], dtype=object)